# How To Run Code On A Real Quantum Computer

**Getting Started With IBM Quantum**


This post appeared first on [Medium](https://towardsdatascience.com/how-to-run-code-on-a-real-quantum-computer-c1fc61ff5b4) and my weekly [email-course](https://pyqml.substack.com/p/how-to-run-code-on-a-real-quantum?s=w).

Quantum computers are no science fiction anymore. They exist. And we can use them — for free. For instance, IBM offers access to their cloud-based quantum computers.

First, we need to [register](https://quantum-computing.ibm.com/) for IBM's quantum computing services. Click on "Create an IBMid account" and provide your details. Once you have verified your email address, you're good to go!

![](./assets/ibmq_dashboard.png)

The Dashboard provides an overview of your booked IBM Quantum Services. Most importantly, it depicts "Your Systems." Click on "View All" and then select "Your Systems" in the filter drop-down right next to the search bar.

You get an overview of the systems you can access. These are a few systems with a handful of qubits. But these are enough to get started.

![](./assets/ibmq_services.png)

When you click on a system, you'll get a more detailed view of its architecture and configuration. For instance, the graphic at the bottom right shows how the system's qubits connect. A limitation you have to cope with when working with a real quantum computer is that we can only entangle connected qubits with each other.

![](./assets/ibmq_quito.png)

So, let's say we want to run our quantum circuit on the 5-qubit Quito device.

Of course, we need to create our circuit first. Usually, we do this in a Jupyter notebook. If you do not have Jupyter up and running yet, this post explains how to set up JuypterLab.
So, let's open a new Jupyter notebook.

We start with a few imports and then, we define the circuit.

In [1]:

from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, IBMQ, execute, transpile, Aer, assemble
from qiskit.tools.monitor import job_monitor

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

qr = QuantumRegister(5)
cr = ClassicalRegister(5)

circuit = QuantumCircuit(qr, cr)
circuit.x(qr[0])
circuit.x(qr[1])
circuit.ccx(qr[0], qr[1], qr[2])
circuit.cx(qr[0], qr[1])
circuit.measure(qr, cr)

As you can see, we only entangle connected qubits (lines 14 and 15).

Before executing the code, we need to connect to the Quito device. Therefore, we need to load our account using your token. You can obtain your API token from the IBM Quantum dashboard. It is a very long string. Make sure you keep it secure.

In [2]:
 # replace TOKEN with your API token string (https://quantum-computing.ibm.com/lab/docs/iql/manage/account/ibmq)
IBMQ.save_account("TOKEN", overwrite=True) 
provider = IBMQ.load_account()

Next, we connect to the provider and get the Quito backend. If you want to connect to other backends, you may need to select another hub, group, and project. But IBM will let you know when they grant you access to any other provider.

Then, we need to transpile the circuit. In this step, Qiskit rewrites your code to match the device's requirements. For instance, if we entangled two qubits that are not connected physically, Qiskit would rewrite the circuit to adhere to the hardware requirements.

Finally, we assemble the circuit and send it to the backend that returns a job object.


In [3]:
# Get backend for experiment
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibmq_quito')

# prepare the circuit for the backend
mapped_circuit = transpile(circuit, backend=backend)
qobj = assemble(mapped_circuit, backend=backend, shots=1024)

# execute the circuit
job = backend.run(qobj)

The job object lets you manage your request. For instance, you can obtain its status

In [4]:
job.status()

<JobStatus.QUEUED: 'job is queued'>

Most likely, the job will be queued for quite some time.

Let's have another look at the IBM Quantum dashboard. You should now have an entry in the "Recent Jobs"-list. When you click on it, you get the details. There, you also see when your circuit is scheduled to run. In my case, it takes about half an hour.

![](./assets/ibmq_jobs.png)

Another essential piece of information is the job id. You can also get it programmatically.



In [5]:

job.job_id()

'62261cc931781061062033d5'

Of course, we're interested in the results of the job once completed. But maybe you don't want to wait that long. So, you may want to retrieve the job later.

In [6]:
job = provider.get_backend('ibmq_quito').retrieve_job('61b32368e4d3eb10e15c8b0e')

If you keep your notebook open or retrieve the job later, the job object returns the results once completed.

In [7]:
result = job.result()
counts = result.get_counts()
print(counts)

{'00000': 25, '00001': 50, '10001': 2, '10010': 2, '10100': 1, '10101': 4, '11101': 1, '00010': 36, '00011': 26, '00100': 204, '00101': 601, '00110': 28, '00111': 39, '01001': 1, '01010': 1, '01100': 1, '01101': 2}


The bitstrings indicate the measured values of the qubits (read from right to left). We can see that the upper qubits at positions four and five are not always 0. This may seem strange because we didn't do anything with them. So, they should remain at the default state of 0. Unfortunately, qubits are very sensitive to noise.

Noise and errors are among the biggest problems we have to cope with in quantum computing today. For instance, IBM recently announced a prize totaling $100,000 to simulate a Heisenberg model Hamiltonian for a three-particle system on IBM Quantum's 7-qubit Jakarta system using Trotterization. The main problem to complete this challenge is to cope with the noise.